# Setup stuff

In [ ]:
#=====[ Setup - don't modify ]=====
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import csv
import pandas as pd
import string
from sklearn.ensemble import RandomForestClassifier

from helper_functions import *

import autism_evaluator_structures


# Read and Combine Training Dataset(s)

In [ ]:
dataFilePath = #Point this to your own training sets
f = open(dataFilePath, 'rU')
dictReader = csv.DictReader(f, delimiter='\t')
data = []
for row in dictReader:
    data.append(row)

df1 = pd.DataFrame(data)
df1.shape

In [ ]:
dataFilePath = #Point this to your own training sets
f = open(dataFilePath, 'rU')
dictReader = csv.DictReader(f, delimiter='\t')
data = []
for row in dictReader:
    data.append(row)

df2 = pd.DataFrame(data)
df2.shape

In [ ]:
dataFilePath = #Point this to your own training sets
f = open(dataFilePath, 'rU')
dictReader = csv.DictReader(f, delimiter='\t')
data = []
for row in dictReader:
    data.append(row)

df3 = pd.DataFrame(data)
df3.shape

In [ ]:
df = df1.append(df2).append(df3)
df.shape

Convert some columns to int type

In [ ]:
for column in ['age_years', 'age_months', 'insturment_module']:
    df[column] = pd.to_numeric(df[column])

Filter by age and group into two age groups

In [ ]:
df = df[(df['age_years']>1) &(df['age_years']<7)]
df['age_group'] = demographic_df['age_years'].apply(lambda x: '<=3' if x<4 else '>=4')

df[['age_group', 'outcome', 'age_years']].groupby(['age_group', 'outcome']).count()

Do some sanity checks on the data

In [ ]:
df.columns

In [ ]:
df.groupby(['sample_source','outcome']).count()

In [ ]:
df1 = df[df.instrument_module==1]

In [ ]:
df1.groupby(['sample_source']).count()

In [ ]:
df1.sort_values(by=['age_years']).groupby(['age_years']).count()

In [ ]:
df2 = df[df.insturment_module==2]
df2.groupby(['sample_source']).count()

In [ ]:
df2.sort_values(by=['age_years']).groupby(['age_years', 'outcome']).count()

# Training Data Preprocessing

limit age depending on instrument module

In [ ]:
video_training = [{},{},{}]


In [ ]:
mod1_min_years = 1 if age_as_feature==True else 2
mod2_min_years = 1 if age_as_feature==True else 3

video_training[1]['data'] = df[ (df.instrument_module==1) &  (df.age_years>=mod1_min_years) & (df.age_years<=max_years)].reset_index(drop=True)
video_training[1]['q_columns'] = columns_about(video_training[1]['data'], 'instument_')
video_training[1]['age_columns'] = ['age_years', 'age_months']
video_training[1]['gender_columns'] = ['gender']


video_training[2]['data'] = df[ (df.instrument_module==2) &  (df.age_years>=mod2_min_years) & (df.age_years<=max_years)].reset_index(drop=True)
video_training[2]['q_columns'] = columns_about(video_training[2]['data'], 'instrument_')
video_training[2]['age_columns'] = ['age_years', 'age_months']
video_training[2]['gender_columns'] = ['gender']

In [ ]:
video_training[1]['data'].groupby(['age_years', 'outcome']).count()

In [ ]:
video_training[2]['data'].groupby(['age_years', 'outcome']).count()

In [ ]:
print 'q cols: ', video_training[module]['q_columns']
print 'age cols: ', video_training[module]['columns']
print 'gender cols: ', video_training[module]['gender_columns']
feature_columns = video_training[module]['q_columns'] + video_training[module]['columns'] + video_training[module]['gender_columns']
print 'feature cols: ', feature_columns

map any value outside {0,1,2,3,4} to 'missing'

In [ ]:
if map_features==True:
    for feature in video_training[module]['q_columns']:
        video_training[module]['data'][feature] = video_training[module]['data'][feature].apply(lambda x: x if x in ['0', '1', '2', '3', '4'] else 'missing')

In [ ]:
for feature in feature_columns:
    print feature
    print video_training[module]['data'][feature].value_counts()

we encode different features differently - this is the list features for each encoding type

In [ ]:
discrete_encoding_features = []
one_hot_encoding_features = []
if run_encoding == 'default':
    scalar_encoding_features = video_training[module]['age_columns']
    one_hot_encoding_features = video_training[module]['gender_columns']+video_training[module]['q_columns']
    presence_of_behavior_features = []
elif run_encoding == 'production':
    scalar_encoding_features = video_training[module]['age_columns']
    presence_of_behavior_features = video_training[module]['gender_columns']+video_training[module]['q_columns']
elif run_encoding == 'scalar':
    scalar_encoding_features = video_training[module]['age_columns']+video_training[module]['gender_columns']+video_training[module]['q_columns']
    presence_of_behavior_features = []
else:
    raise ValueError('Error, run_encoding: '+run_encoding+' not understood')


feature_encoding_map = {}
for feature in scalar_encoding_features:
    feature_encoding_map[feature] = 'scalar'
for feature in one_hot_encoding_features:
    feature_encoding_map[feature] = 'one_hot'
for feature in discrete_encoding_features:
    feature_encoding_map[feature] = 'discrete'
for feature in presence_of_behavior_features:
    feature_encoding_map[feature] = 'presence_of_behavior'

print 'for run_desc: ', run_desc, ', feature_encoding_map: ', feature_encoding_map
print 'feature_columns now: ', feature_columns

# Create missing data and inject loss

In [ ]:
### Define loss/missing data instructions
lossColumnsToDo = [[], video_training[1]['q_columns'], video_training[2]['q_columns']]
lossInstructions = MissingDataInstructions.get_missing_instructions_from_SQL(lossColumnsToDo, minProb=0.01, 
                                                                             desc='clinical')

print 'For run_desc: ', run_desc, ', inject_loss: ', inject_loss
if inject_loss is not None and inject_loss=='proportional':
    video_training[module]['data'] = inject_proportional_loss_when_presence_encoding(video_training[module]['data'], outcome_key='outcome',
                    instructions=None, missing_value='missing', prior_autism_frac=0.5, module=module, validation=True)

for feature in feature_columns:
    print feature
    print video_training[module]['data'][feature].value_counts()

# Video Model Training

In [ ]:
import time
date_string = time.strftime("%-m.%-d.%y")

output_directory = 'output/'
filename_prefix = "Instrument"+str(module)+"."+run_desc+'_'+date_string

These are ML training constants

In [ ]:
outcome_column = 'outcome'
outcome_classes = ['autism','not']

outcome_class_priors =  [(1.0/2.0), (1.0/2.0)]       # IN CLINICAL CENTRES

## Build a model with all data as a warm-up

In [ ]:
n_estimators = 200
criterion = "gini"
max_features = 'log2'
max_depth=5
class_weight = None

dunno_range = (0.2,0.9)

number_of_features_to_keep = 15

#sprinkle some random features
video_training[module]['data']['random1'] = np.random.choice(3, len(video_training[module]['data']), p=[0.1, 0.6, 0.3])
video_training[module]['data']['random2'] = np.random.choice(4, len(video_training[module]['data']), p=[0.25, 0.25, 0.25, 0.25])
video_training[module]['data']['random3'] = np.random.choice(2, len(video_training[module]['data']), p=[0.6, 0.4])


sample_weights = balance_dataset_on_dimensions(video_training[module]['data'], balance_dimensions, verbose=False)

model, features, y_predicted_without_dunno, y_predicted_with_dunno, y_predicted_probs =\
                    all_data_model(video_training[module]['data'], feature_columns,
                    feature_encoding_map, outcome_column, sample_weights, dunno_range,
                    RandomForestClassifier,  n_estimators = n_estimators, criterion = criterion,
                    max_features = max_features, class_weight = class_weight, max_depth=max_depth)
  
important_features = get_important_features(model, features, 0.001)
    
metrics = get_classifier_performance_metrics(outcome_classes, outcome_class_priors,
            video_training[module]['data'][outcome_column], y_predicted_without_dunno, y_predicted_with_dunno, y_predicted_probs)
print 'metrics: ', metrics
for feature in important_features:
    print(feature)
    print("\n") 
print_classifier_performance_metrics(outcome_classes, metrics)

top_feature_columns =  get_best_features(important_features, number_of_features_to_keep, ['=', '_behavior_present'], [])
n_features = 10 if module==1 else 9
default_top_N_features = cp.deepcopy(top_feature_columns[:n_features])

print 'for run_desc: ', run_desc, ', top features: ', default_top_N_features

## Build a model repeatedly and keep tabs on which features get used most often

In [ ]:
from sklearn.ensemble import RandomForestClassifier

n_estimators = 200
criterion = "gini"
max_features = 'log2'
class_weight = None
max_depth=5


dunno_range = (0.2,0.9)


feature_tally = {}
number_of_features_to_keep = 15

#do the following many times
number_of_tries = 10


for i in range(0,number_of_tries):
    print 'Starting try ', i, ' out of ', number_of_tries
    
    #grab a random subsample
    dataset_for_this_try = subsample_per_class(video_training[module]['data'], outcome_column, {'autism':0.9, 'not':0.9} )
    
    #sprinkle some random features
    dataset_for_this_try['random1'] = np.random.choice(3, len(dataset_for_this_try), p=[0.1, 0.6, 0.3])
    dataset_for_this_try['random2'] = np.random.choice(4, len(dataset_for_this_try), p=[0.25, 0.25, 0.25, 0.25])
    dataset_for_this_try['random3'] = np.random.choice(2, len(dataset_for_this_try), p=[0.6, 0.4])
    
    sample_weights_for_this_try = balance_dataset_on_dimensions(dataset_for_this_try,
                                                balance_dimensions, verbose=False)

    model, features, y_predicted_without_dunno, y_predicted_with_dunno, y_predicted_probs =\
        all_data_model(dataset_for_this_try, feature_columns+['random1','random2','random3'], 
        feature_encoding_map, outcome_column, sample_weights_for_this_try, dunno_range, RandomForestClassifier,  
        n_estimators = n_estimators, criterion = criterion, max_features = max_features,
        class_weight = class_weight, max_depth=max_depth)
    
    important_features = get_important_features(model, features, 0.01)
    
    top_feature_columns = get_best_features(important_features, number_of_features_to_keep,
                                            ['=', '_behavior_present'], ['gender','age_months','age_years'])
    for feature in top_feature_columns:
        if feature in feature_tally:
            feature_tally[feature]+=1
        else:
            feature_tally[feature]=1

tally = sorted(feature_tally.items(), key=lambda pair: pair[1], reverse=True)
print tally



### How statistically limited are we?

In [ ]:
fracs_to_check = np.array(list(np.arange(0.01, 0.1, 0.01)) +\
                list(np.arange(0.1, 0.5, 0.05)) +\
                list(np.arange(0.5, 1., 0.1)) + [1.])
n_duplicate_runs=20
n_folds=5

plot_title=''

training_data_statistical_stability_tests(video_training[module]['data'], sample_frac_sizes=fracs_to_check, feature_columns=feature_columns,
                  feature_encoding_map=feature_encoding_map, target_column=outcome_column,
                  sample_weights=sample_weights, dunno_range=dunno_range, model_function=RandomForestClassifier,
                  outcome_classes=outcome_classes, outcome_class_priors=outcome_class_priors,
                  cross_validate_group_id='unique_patient_id', n_folds=n_folds, n_duplicate_runs=n_duplicate_runs,
                  do_plotting=True, plot_title=plot_title, n_estimators = n_estimators, criterion = criterion,
                  max_features = max_features, class_weight = class_weight, max_depth=max_depth)

## Now we cross validate to gauge model performance

In [ ]:
from sklearn.ensemble import RandomForestClassifier

n_estimators = 200
criterion = "gini"
max_features = 'log2'
class_weight = None
max_depth=5
#cross_validate_with_groupid=None
cross_validate_with_groupid='unique_patient_id'

class_weight = None

dunno_range = (0.2,0.9)
n_folds = 20

print 'For module: ', module
print 'Using following parameters:'
print 'criterion: ', criterion
print 'max_features: ', max_features
print 'max_depth: ', max_depth
print 'n_estimators: ', n_estimators
print 'features: ', feature_columns

output = cross_validate_model(video_training[module]['data'], sample_weights, feature_columns,
                feature_encoding_map, outcome_column, dunno_range, n_folds, outcome_classes, 
                outcome_class_priors, RandomForestClassifier,  groupid=cross_validate_with_groupid, 
                n_estimators = n_estimators, criterion = criterion, max_features = max_features,
                class_weight = class_weight, max_depth=max_depth)

print_classifier_performance_metrics(outcome_classes, output['overall_metrics'])



#### Grid search for best decision forest model parameters

In [ ]:
print 'About to run grid search for ', run_desc, ', module: ', module
print 'ages in dataset: ', video_training[module]['data']['age_years'].value_counts()
print 'balance on: ', balance_dimensions
print 'Features to use in grid search:'
for feature in feature_columns:
    print feature, ', encoding: ', feature_encoding_map[feature], ', values: ', video_training[module]['data'][feature].value_counts()

n_autism = len(video_training[module]['data'][video_training[module]['data']['outcome']=='autism'].index)
n_not = len(video_training[module]['data'][video_training[module]['data']['outcome']=='not'].index)
print 'n_autism: ', n_autism
print 'n_not: ', n_not
print 'n_total: ', len(video_training[module]['data'].index)
assert n_autism + n_not == len(video_training[module]['data'].index)

#this is where we output to
output_filename = output_directory+filename_prefix+".gridSearch.modelParams.csv"

criterion = ["entropy"]
max_features = ['log2']
max_depth = [4,5,6,7]
n_estimators = [200]

#cross_validate_with_groupid=None
cross_validate_with_groupid='unique_patient_id'

param_combinations = get_combinations([criterion, max_features, max_depth, n_estimators])

#these are bootstrapping parameters
bootstrapping_number_of_tries = 10     #run every node in the grid search this many times and average out the resulting metrics
bootstrapping_sample_percent = 0.9    #for every run, random-sample this percentage of the dataset (stratified by target class) 


#these static params are problem specific
n_folds = 20

#this dunno range param is outside of the ML model so let's fix it to some convenient values for now
dunno_range = (0.2,0.9)

print 'For module: ', module
print 'Do grid search for best model parameters using:'
print 'features: ', feature_columns
sys.stdout.flush()

def modeling_function(param_combination):
    def sampling_function_per_try(dataset):
        sample = subsample_per_class(dataset, outcome_column, {'autism':bootstrapping_sample_percent, 'not':bootstrapping_sample_percent})
        return sample
    def ml_function_per_try(dataset_per_try):
        sample_weights_per_try = balance_dataset_on_dimensions(dataset_per_try, balance_dimensions)

        metrics = cross_validate_model(dataset_per_try, sample_weights_per_try,
                feature_columns, feature_encoding_map, outcome_column, dunno_range,
                n_folds, outcome_classes, outcome_class_priors, RandomForestClassifier, groupid=cross_validate_with_groupid,
                criterion = param_combination[0], max_features = param_combination[1],
                max_depth=param_combination[2], n_estimators = param_combination[3])
        return metrics['overall_metrics']
    averaged_metrics, averaged_metrics_err =  bootstrap(video_training[module]['data'], bootstrapping_number_of_tries, sampling_function_per_try,
                                  ml_function_per_try, return_errs=True, verbose=False)
    print 'For param_combination: ', param_combination, ', AUC: ', averaged_metrics['without_dunno']['auc']
    sys.stdout.flush()
    return averaged_metrics, averaged_metrics_err



reporting_function = lambda param_combination, (averaged_metrics, averaged_metrics_err): [ averaged_metrics['without_dunno']['auc'],
                                                                  averaged_metrics_err['without_dunno']['auc'],
                                                                  averaged_metrics['without_dunno']['dataset_precision_per_class']['autism'],
                                                                  averaged_metrics['without_dunno']['reallife_precision_per_class']['autism'],
                                                                  averaged_metrics['without_dunno']['dataset_recall_per_class']['autism'],
                                                                  averaged_metrics_err['without_dunno']['dataset_recall_per_class']['autism'],
                                                                  averaged_metrics['without_dunno']['dataset_precision_per_class']['not'],
                                                                  averaged_metrics['without_dunno']['reallife_precision_per_class']['not'],
                                                                  averaged_metrics['without_dunno']['dataset_recall_per_class']['not'], 
                                                                  averaged_metrics_err['without_dunno']['dataset_recall_per_class']['not'],
                                                                  n_autism,
                                                                  n_not,
                                                                 ]


#run grid search
report = grid_search(modeling_function, param_combinations, reporting_function)
    
#write outputs to file
output_file = open(output_filename,'w')
header = ','.join(['criterion','max_features', 'max_depth', 'n_trees','AUC','AUC err', 'autism precision [Dataset]', 
                   'autism precision [Reallife]', 'autism recall', 'autism recall err', 'not precision [Dataset]',
                   'not precision [Reallife]', 'not recall', 'not recall err', 'n_autism', 'n_not'])
output_file.write(header+"\n")
for line in report:
    output_file.write(','.join([str(x) for x in line]))
    output_file.write("\n")
output_file.close()

print 'Grid search done for run_desc: ', run_desc



## Grid search for best dunno range

In [ ]:
balance_dimensions#this is where we output to
output_filename = output_directory+filename_prefix+".gridSearch.dunnoRange.csv"


#these are bootstrapping parameters
bootstrapping_number_of_tries = 10     #run every node in the grid search this many times and average out the resulting metrics
bootstrapping_sample_percent = 0.9    #for every run, random-sample this percentage of the dataset (stratified by target class) 

#cross_validate_with_groupid=None
cross_validate_with_groupid='unique_patient_id'


#these static params are problem specific
n_folds = 20

#these params are outside of the ML model so let's fix them to some convenient values for now
dunno_range_min = [0.01, 0.1, 0.2, 0.3, 0.4, 0.45, 0.49]
dunno_range_max = [0.51, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]
param_combinations = get_combinations([dunno_range_min, dunno_range_max])



def modeling_function(param_combination):
    def sampling_function_per_try(dataset):
        sample = subsample_per_class(dataset, outcome_column, {'autism':bootstrapping_sample_percent,
                                                'not':bootstrapping_sample_percent})
        return sample
    def ml_function_per_try(dataset_per_try):
        print 'Running param_combination: ', param_combination
        sys.stdout.flush()
        sample_weights_per_try = balance_dataset_on_dimensions(dataset_per_try, balance_dimensions)

        metrics = cross_validate_model(dataset_per_try, sample_weights_per_try,
                    feature_columns, feature_encoding_map, outcome_column,
                    (param_combination[0], param_combination[1]), n_folds, outcome_classes,
                    outcome_class_priors, RandomForestClassifier, groupid=cross_validate_with_groupid, criterion = criterion,
                    max_features = max_features,  max_depth = max_depth, n_estimators = n_estimators)
        return metrics['overall_metrics']
    
    averaged_metrics =  bootstrap(video_training[module]['data'], bootstrapping_number_of_tries, sampling_function_per_try, ml_function_per_try)
    print 'For param_combination: ', param_combination, ', AUC: ', averaged_metrics['without_dunno']['auc'], ' +/- ',\
                 averaged_metrics_err['without_dunno']['auc']
    sys.stdout.flush()
    return averaged_metrics



reporting_function = lambda param_combination, averaged_metrics: [ averaged_metrics['with_dunno']['dataset_classification_rate'],
                                                          averaged_metrics['with_dunno']['reallife_classification_rate'],
                                                          averaged_metrics['with_dunno']['dataset_precision_per_class']['autism'],
                                                          averaged_metrics['with_dunno']['reallife_precision_per_class']['autism'],
                                                          averaged_metrics['with_dunno']['dataset_recall_per_class']['autism'],
                                                          averaged_metrics['with_dunno']['dataset_precision_per_class']['not'],
                                                          averaged_metrics['with_dunno']['reallife_precision_per_class']['not'],
                                                          averaged_metrics['with_dunno']['dataset_recall_per_class']['not']
                                                        ]




#run grid search
report = grid_search(modeling_function, param_combinations, reporting_function)

#write outputs to file
output_file = open(output_filename,'w')
header = ','.join(['dunno_range_min','dunno_range_max', 
                'classification rate [Dataset]',
                'classification rate [Reallife]',
                'autism precision [Dataset]', 
                'autism precision [Reallife]', 
                'autism recall', 
                'not precision [Dataset]', 
                'not precision [Reallife]', 
                'not recall'])
output_file.write(header+"\n")
report = grid_search(modeling_function, param_combinations, reporting_function)
for line in report:
    output_file.write(','.join([str(x) for x in line]))
    output_file.write("\n")
output_file.close()




## Use optimal model parameters and optimal dunno range to cross validate model

In [ ]:
print 'For module: ', module
print 'Using following parameters:'
print 'criterion: ', criterion
print 'max_features: ', max_features
print 'max_depth: ', max_depth
print 'n_estimators: ', n_estimators
print 'features: ', feature_columns

output = cross_validate_model(video_training[module]['data'], sample_weights, feature_columns, feature_encoding_map,
                                       outcome_column, dunno_range, n_folds, outcome_classes, outcome_class_priors,
                                       RandomForestClassifier,  groupid=cross_validate_with_groupid, n_estimators = n_estimators, 
                                       criterion = criterion, max_features = max_features, class_weight = class_weight,
                                      max_depth=max_depth)

print_classifier_performance_metrics(outcome_classes, output['overall_metrics'])



## Now build the final model using all corresponding samples

In [ ]:
#this is where we output to
output_filename = output_directory+run_choices+"_"+filename_prefix+".model"
print 'For module: ', module
print 'Using following parameters:'
print 'criterion: ', criterion
print 'max_features: ', max_features
print 'max_depth: ', max_depth
print 'n_estimators: ', n_estimators
print 'feature_columns: ', feature_columns


#build model
model, features, y_predicted_without_dunno, y_predicted_with_dunno, y_predicted_probs =\
           all_data_model(video_training[module]['data'], feature_columns, feature_encoding_map, outcome_column, 
           sample_weights, dunno_range, RandomForestClassifier,  n_estimators = n_estimators, criterion = criterion,
           max_features = max_features, max_depth=max_depth, class_weight = class_weight)


#save features into a separate file
output_filename = output_directory+filename_prefix+".features.txt"
output_file = open(output_filename,'w')
ordered_features = sorted(zip(features, model.feature_importances_), key=lambda x: x[1], reverse=True)
output_file.write("QUESTIONS BY IMPORTANCE:\n\n")
written_already = []
for feature in [x[0].split('=')[0] for x in ordered_features]:
    if feature not in written_already:
        written_already += [feature]
        output_file.write(feature)
        output_file.write("\n")
output_file.write("\n\nFEATURES BY IMPORTANCE:\n\n")
for pair in ordered_features:
    output_file.write(str(pair[0])+"\t"+str(pair[1]))
    output_file.write("\n")
output_file.close()